In [4]:
# pip install torch torch-geometric pandas numpy scikit-learn

In [5]:
import pandas as pd
import numpy as np
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

In [7]:
# Load and preprocess data (reusing existing functions)
def load_and_preprocess_data(file_path, columns):
    df = (pd.read_csv(file_path, usecols=columns)
          .fillna("empty")
          .query("DevType != 'empty' and DevType != 'Other (please specify):'")
          .reset_index(drop=True))
    return df

def combine_skills(df, columns, new_column_name):
    df[new_column_name] = (df[columns]
                           .fillna('')
                           .agg(';'.join, axis=1)
                           .apply(lambda x: x.split(';') if x else []))
    return df

# Load and preprocess data
columns_finalized = ['ResponseId', 'DevType', 'LanguageHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'PlatformHaveWorkedWith',
                     'MiscTechHaveWorkedWith', 'WebframeHaveWorkedWith', 'DatabaseHaveWorkedWith']

df = load_and_preprocess_data("/kaggle/input/survey-results/survey_results_public.csv", columns_finalized)
df = combine_skills(df, ['LanguageHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'PlatformHaveWorkedWith',
                         'MiscTechHaveWorkedWith', 'WebframeHaveWorkedWith', 'DatabaseHaveWorkedWith'], 
                    'AllSkillsHaveWorkedWith')

# Filter out students
df_non_students = df.query("DevType != 'Student'").reset_index(drop=True)

# Prepare data for GNN
skills = df_non_students['AllSkillsHaveWorkedWith'].explode().unique()
skill_encoder = LabelEncoder().fit(skills)
dev_type_encoder = LabelEncoder().fit(df_non_students['DevType'])

# Define a fixed feature size
feature_size = 64

# Create node features
num_users = len(df_non_students)
num_skills = len(skills)
num_dev_types = len(dev_type_encoder.classes_)

user_features = torch.randn(num_users, feature_size)
skill_features = torch.randn(num_skills, feature_size)
dev_type_features = torch.randn(num_dev_types, feature_size)


# Create edges
user_skill_edges = []
user_dev_type_edges = []

for idx, row in df_non_students.iterrows():
    user_skills = skill_encoder.transform(row['AllSkillsHaveWorkedWith'])
    user_skill_edges.extend([(idx, skill_idx + num_users) for skill_idx in user_skills])
    user_dev_type_edges.append((idx, dev_type_encoder.transform([row['DevType']])[0] + num_users + num_skills))

# Combine all edges
edges = user_skill_edges + user_dev_type_edges
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()


# Combine all node features
x = torch.cat([user_features, skill_features, dev_type_features], dim=0)



# Create PyTorch Geometric Data object
data = Data(x=x, edge_index=edge_index)

# Define GNN model
# class GNNJobRecommender(torch.nn.Module):
#     def __init__(self, num_features, hidden_channels, num_classes):
#         super(GNNJobRecommender, self).__init__()
#         self.conv1 = GCNConv(num_features, hidden_channels)
#         self.conv2 = GCNConv(hidden_channels, num_classes)

#     def forward(self, x, edge_index):
#         x = self.conv1(x, edge_index)
#         x = x.relu()
#         x = self.conv2(x, edge_index)
#         return x

from torch_geometric.nn import GATConv

class GNNJobRecommender(torch.nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GNNJobRecommender, self).__init__()
        # 8 attention heads in the first layer
        self.conv1 = GATConv(num_features, hidden_channels, heads=8, dropout=0.6)
        # 1 attention head in the second layer
        self.conv2 = GATConv(hidden_channels * 8, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        return x

# Initialize model
model = GNNJobRecommender(num_features=feature_size, hidden_channels=32, num_classes=num_dev_types)

# Split data into train and test sets
train_mask = torch.zeros(num_users, dtype=torch.bool)
train_mask[:num_users-5000] = True
test_mask = ~train_mask

# Prepare labels
y = torch.tensor(dev_type_encoder.transform(df_non_students['DevType']))

# Training function
def train(data, model, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = torch.nn.functional.cross_entropy(out[:num_users][train_mask], y[train_mask])
    loss.backward()
    optimizer.step()
    return loss

# Evaluation function
def test(data, model):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out[:num_users].argmax(dim=1)
    test_correct = pred[test_mask] == y[test_mask]
    test_acc = int(test_correct.sum()) / int(test_mask.sum())
    return test_acc

# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(50):
    loss = train(data, model, optimizer)
    if epoch % 10 == 0:
        test_acc = test(data, model)
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Test Acc: {test_acc:.4f}')

# Function to get job recommendations
def get_job_recommendations(user_idx, model, data, dev_type_encoder, top_n=5):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        user_probs = torch.nn.functional.softmax(out[user_idx], dim=0)
        top_indices = user_probs.argsort(descending=True)[:top_n]
        return dev_type_encoder.inverse_transform(top_indices.numpy())

# Create a boolean mask for test users
test_mask_np = test_mask.numpy()

# Example: Get top 5 job recommendations for a specific user in the test set
test_user_idx = np.where(test_mask_np)[0][0]
recommended_jobs = get_job_recommendations(test_user_idx, model, data, dev_type_encoder)
print(f"Top job recommendations for user {test_user_idx}: {recommended_jobs}")

# Calculate MRR for test set
def calculate_mrr(true_jobs, predicted_jobs):
    mrr = 0.0
    for true_job, pred_jobs in zip(true_jobs, predicted_jobs):
        try:
            rank = np.where(pred_jobs == true_job)[0][0] + 1
            mrr += 1 / rank
        except IndexError:
            continue
    return mrr / len(true_jobs)

Epoch: 000, Loss: 4.4361, Test Acc: 0.1878
Epoch: 010, Loss: 3.1584, Test Acc: 0.3372
Epoch: 020, Loss: 2.9893, Test Acc: 0.3442
Epoch: 030, Loss: 2.8965, Test Acc: 0.3434
Epoch: 040, Loss: 2.8257, Test Acc: 0.3430
Top job recommendations for user 46885: ['Developer, full-stack' 'Developer, back-end' 'Developer, front-end'
 'Developer, desktop or enterprise applications' 'Developer, mobile']


In [8]:
from tqdm import tqdm  # Ensure you have tqdm for progress bar

# Precompute the softmax probabilities for all users
def get_job_recommendations_batch(data, model):
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        softmax_out = torch.nn.functional.softmax(out[:num_users], dim=1)  # Precompute softmax for all users
    return softmax_out

# Precompute predictions for all users at once
all_user_probs = get_job_recommendations_batch(data, model)

# Get top N job recommendations for a specific user
def get_top_n_jobs(user_probs, dev_type_encoder, top_n=5):
    top_indices = user_probs.argsort(descending=True)[:top_n]
    return dev_type_encoder.inverse_transform(top_indices.numpy())

# Precompute predicted jobs for all test users
true_jobs_test = df_non_students.loc[test_mask_np, 'DevType'].values
test_user_indices = np.where(test_mask_np)[0]
predicted_jobs_test = []
for user_idx in tqdm(test_user_indices):
    user_probs = all_user_probs[user_idx]
    recommended_jobs = get_top_n_jobs(user_probs, dev_type_encoder, top_n=10)
    predicted_jobs_test.append(recommended_jobs)

# Calculate MRR for the test set
mrr_score_test = calculate_mrr(true_jobs_test, predicted_jobs_test)
print(f'MRR Score on Test Set: {mrr_score_test}')

100%|██████████| 5000/5000 [00:00<00:00, 5268.69it/s]

MRR Score on Test Set: 0.4969627777777765


In [24]:
import pandas as pd
import numpy as np
import torch
import torch_geometric
from torch_geometric.data import HeteroData
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import torch
import torch.nn.functional as F

def load_and_preprocess_data(file_path, columns):
    """Load and preprocess the survey data."""
    df = pd.read_csv(file_path, usecols=columns)
    # Convert string columns to lists if they're semicolon-separated
    for col in columns:
        if col not in ['ResponseId', 'DevType']:
            df[col] = df[col].fillna('')
    df = df.dropna(subset=['DevType'])  # Ensure DevType is not null
    return df

def combine_skills(df, skill_columns, output_column):
    """Combine multiple skill columns into a single list."""
    def split_skills(x):
        skills = []
        for col in x:
            if pd.notna(col) and col:
                skills.extend([s.strip() for s in str(col).split(';') if s.strip()])
        return skills

    df[output_column] = df[skill_columns].apply(split_skills, axis=1)
    return df

def prepare_heterogeneous_data(df_non_students, feature_size=64):
    """Prepare heterogeneous graph data."""
    # Handle empty skill lists
    df_non_students = df_non_students[df_non_students['AllSkillsHaveWorkedWith'].apply(len) > 0].reset_index(drop=True)
    
    # Prepare encoders
    all_skills = set()
    for skills in df_non_students['AllSkillsHaveWorkedWith']:
        all_skills.update(skills)
    skills = np.array(list(all_skills))
    
    skill_encoder = LabelEncoder()
    skill_encoder.fit(skills)
    
    dev_type_encoder = LabelEncoder()
    dev_type_encoder.fit(df_non_students['DevType'])
    
    # Create HeteroData object
    data = HeteroData()
    
    # Add node features
    num_users = len(df_non_students)
    num_skills = len(skills)
    num_dev_types = len(dev_type_encoder.classes_)
    
    # Initialize features with normalization
    data['user'].x = torch.nn.functional.normalize(torch.randn(num_users, feature_size), p=2, dim=1)
    data['skill'].x = torch.nn.functional.normalize(torch.randn(num_skills, feature_size), p=2, dim=1)
    data['dev_type'].x = torch.nn.functional.normalize(torch.randn(num_dev_types, feature_size), p=2, dim=1)
    
    # Create edges
    user_skill_src = []
    user_skill_dst = []
    for idx, skills in enumerate(df_non_students['AllSkillsHaveWorkedWith']):
        if skills:  # Check if skills list is not empty
            skill_indices = skill_encoder.transform(skills)
            user_skill_src.extend([idx] * len(skill_indices))
            user_skill_dst.extend(skill_indices)
    
    dev_type_skill_src = []
    dev_type_skill_dst = []
    for dev_type_idx, dev_type in enumerate(dev_type_encoder.classes_):
        dev_type_mask = df_non_students['DevType'] == dev_type
        dev_type_users = df_non_students[dev_type_mask]
        if not dev_type_users.empty:
            dev_type_skills = []
            for skills in dev_type_users['AllSkillsHaveWorkedWith']:
                dev_type_skills.extend(skills)
            if dev_type_skills:
                skill_counts = pd.Series(dev_type_skills).value_counts()
                top_skills = skill_counts.nlargest(min(10, len(skill_counts))).index
                dev_type_skill_indices = skill_encoder.transform(top_skills)
                dev_type_skill_src.extend([dev_type_idx] * len(dev_type_skill_indices))
                dev_type_skill_dst.extend(dev_type_skill_indices)
    
    user_dev_type_src = list(range(len(df_non_students)))
    user_dev_type_dst = dev_type_encoder.transform(df_non_students['DevType'])
    
    # Convert to tensors and add edges to data object
    if user_skill_src:  # Only create edges if there are any
        data['user', 'has_skill', 'skill'].edge_index = torch.tensor([user_skill_src, user_skill_dst])
        data['skill', 'rev_has_skill', 'user'].edge_index = torch.tensor([user_skill_dst, user_skill_src])
    
    if dev_type_skill_src:  # Only create edges if there are any
        data['dev_type', 'requires_skill', 'skill'].edge_index = torch.tensor([dev_type_skill_src, dev_type_skill_dst])
        data['skill', 'rev_requires_skill', 'dev_type'].edge_index = torch.tensor([dev_type_skill_dst, dev_type_skill_src])
    
    data['user', 'has_type', 'dev_type'].edge_index = torch.tensor([user_dev_type_src, user_dev_type_dst])
    data['dev_type', 'rev_has_type', 'user'].edge_index = torch.tensor([user_dev_type_dst, user_dev_type_src])
    
    return data, skill_encoder, dev_type_encoder, df_non_students

class ImprovedHGNNJobRecommender(torch.nn.Module):
    def __init__(self, hidden_channels, num_classes):
        super(ImprovedHGNNJobRecommender, self).__init__()
        
        self.conv1 = HeteroConv({
            ('user', 'has_skill', 'skill'): GCNConv(-1, hidden_channels),
            ('skill', 'rev_has_skill', 'user'): GCNConv(-1, hidden_channels),
            ('dev_type', 'requires_skill', 'skill'): GCNConv(-1, hidden_channels),
            ('skill', 'rev_requires_skill', 'dev_type'): GCNConv(-1, hidden_channels),
            ('user', 'has_type', 'dev_type'): GCNConv(-1, hidden_channels),
            ('dev_type', 'rev_has_type', 'user'): GCNConv(-1, hidden_channels),
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('user', 'has_skill', 'skill'): GCNConv(hidden_channels, hidden_channels),
            ('skill', 'rev_has_skill', 'user'): GCNConv(hidden_channels, hidden_channels),
            ('dev_type', 'requires_skill', 'skill'): GCNConv(hidden_channels, hidden_channels),
            ('skill', 'rev_requires_skill', 'dev_type'): GCNConv(hidden_channels, hidden_channels),
            ('user', 'has_type', 'dev_type'): GCNConv(hidden_channels, hidden_channels),
            ('dev_type', 'rev_has_type', 'user'): GCNConv(hidden_channels, hidden_channels),
        }, aggr='mean')
        
        self.lin = torch.nn.Linear(hidden_channels, num_classes)

    def forward(self, data):
        x_dict, edge_index_dict = data.x_dict, data.edge_index_dict
        
        # First convolution layer
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: F.relu(x) for key, x in x_dict.items()}
        
        # Second convolution layer
        x_dict = self.conv2(x_dict, edge_index_dict)
        x_dict = {key: F.relu(x) for key, x in x_dict.items()}
        
        # Final layer for classification
        return self.lin(x_dict['user'])

def get_job_recommendations_batch(model, data):
    """Get job recommendations for all users in batch."""
    model.eval()
    with torch.no_grad():
        logits = model(data)
        return F.softmax(logits, dim=1).numpy()

def get_top_n_jobs(user_probs, dev_type_encoder, top_n=10):
    """Get top N job recommendations for a user."""
    top_indices = np.argsort(user_probs)[-top_n:][::-1]
    return dev_type_encoder.inverse_transform(top_indices)

def calculate_mrr(true_jobs, predicted_jobs):
    """Calculate Mean Reciprocal Rank."""
    reciprocal_ranks = []
    for true_job, predictions in zip(true_jobs, predicted_jobs):
        try:
            rank = predictions.tolist().index(true_job) + 1
            reciprocal_ranks.append(1.0 / rank)
        except ValueError:
            reciprocal_ranks.append(0.0)
    return np.mean(reciprocal_ranks)

def train_with_early_stopping(model, data, optimizer, num_users, train_mask, val_mask, labels, 
                            patience=5, epochs=100, batch_size=32):
    """
    Train the GNN model with early stopping based on validation loss.
    
    Parameters:
    - model: The GNN model to be trained
    - data: HeteroData object containing the graph
    - optimizer: PyTorch optimizer
    - num_users: Total number of users
    - train_mask: Boolean mask for training samples
    - val_mask: Boolean mask for validation samples
    - labels: Ground truth labels
    - patience: Number of epochs to wait before early stopping
    - epochs: Maximum number of training epochs
    - batch_size: Batch size for training
    
    Returns:
    - Trained model
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    data = data.to(device)
    labels = labels.to(device)
    
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    train_losses = []
    val_losses = []
    
    # Get indices for training and validation
    train_indices = torch.where(train_mask)[0]
    val_indices = torch.where(val_mask)[0]
    
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        num_batches = 0
        
        # Training
        for i in range(0, len(train_indices), batch_size):
            batch_indices = train_indices[i:i + batch_size]
            
            # Zero gradients
            optimizer.zero_grad()
            
            # Forward pass
            out = model(data)
            batch_out = out[batch_indices]
            batch_labels = labels[batch_indices]
            
            # Calculate loss
            loss = F.cross_entropy(batch_out, batch_labels)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            num_batches += 1
        
        avg_train_loss = total_train_loss / num_batches
        train_losses.append(avg_train_loss)
        
        # Validation
        model.eval()
        total_val_loss = 0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            # Forward pass on validation set
            val_out = model(data)
            val_batch_out = val_out[val_indices]
            val_batch_labels = labels[val_indices]
            
            # Calculate validation loss
            val_loss = F.cross_entropy(val_batch_out, val_batch_labels)
            total_val_loss = val_loss.item()
            
            # Calculate validation accuracy
            _, predicted = torch.max(val_batch_out, 1)
            val_correct = (predicted == val_batch_labels).sum().item()
            val_total = len(val_indices)
        
        avg_val_loss = total_val_loss
        val_accuracy = val_correct / val_total
        val_losses.append(avg_val_loss)
        
        # Print epoch statistics
        print(f'Epoch [{epoch+1}/{epochs}]')
        print(f'Training Loss: {avg_train_loss:.4f}')
        print(f'Validation Loss: {avg_val_loss:.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        print('-' * 50)
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            
        # Early stopping trigger
        if patience_counter >= patience:
            print(f'Early stopping triggered after {epoch + 1} epochs')
            break
    
    # Load best model state
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model

def train_epoch(model, data, optimizer, train_indices, labels, batch_size):
    """Helper function to train one epoch"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    # Create batches
    for i in range(0, len(train_indices), batch_size):
        batch_indices = train_indices[i:i + batch_size]
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        out = model(data)
        batch_out = out[batch_indices]
        batch_labels = labels[batch_indices]
        
        # Calculate loss
        loss = F.cross_entropy(batch_out, batch_labels)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    return total_loss / num_batches

def evaluate(model, data, indices, labels):
    """Helper function to evaluate the model"""
    model.eval()
    with torch.no_grad():
        out = model(data)
        batch_out = out[indices]
        batch_labels = labels[indices]
        
        # Calculate loss
        loss = F.cross_entropy(batch_out, batch_labels)
        
        # Calculate accuracy
        _, predicted = torch.max(batch_out, 1)
        correct = (predicted == batch_labels).sum().item()
        total = len(indices)
        
    return loss.item(), correct / total

if __name__ == "__main__":
    # Load and preprocess data
    columns_finalized = ['ResponseId', 'DevType', 'LanguageHaveWorkedWith', 'ToolsTechHaveWorkedWith', 
                        'PlatformHaveWorkedWith', 'MiscTechHaveWorkedWith', 'WebframeHaveWorkedWith', 
                        'DatabaseHaveWorkedWith']
    
    df = load_and_preprocess_data("survey_results_public.csv", columns_finalized)
    df = combine_skills(df, ['LanguageHaveWorkedWith', 'ToolsTechHaveWorkedWith', 'PlatformHaveWorkedWith',
                            'MiscTechHaveWorkedWith', 'WebframeHaveWorkedWith', 'DatabaseHaveWorkedWith'], 
                       'AllSkillsHaveWorkedWith')
    
    # Filter out students and empty skills
    df_non_students = df.query("DevType != 'Student'").reset_index(drop=True)
    
    # Prepare data
    data, skill_encoder, dev_type_encoder, df_non_students = prepare_heterogeneous_data(df_non_students)
    
    # Prepare masks
    num_users = len(df_non_students)
    indices = np.random.permutation(num_users)
    train_idx = indices[:int(0.7*num_users)]
    val_idx = indices[int(0.7*num_users):int(0.85*num_users)]
    test_idx = indices[int(0.85*num_users):]
    
    train_mask = torch.zeros(num_users, dtype=torch.bool)
    val_mask = torch.zeros(num_users, dtype=torch.bool)
    test_mask = torch.zeros(num_users, dtype=torch.bool)
    
    train_mask[train_idx] = True
    val_mask[val_idx] = True
    test_mask[test_idx] = True
    
    # Prepare labels
    y = torch.tensor(dev_type_encoder.transform(df_non_students['DevType'].values))
    
    # Initialize model
    num_classes = len(dev_type_encoder.classes_)
    model = ImprovedHGNNJobRecommender(hidden_channels=32, num_classes=num_classes)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    
    # Train model
    model = train_with_early_stopping(model, data, optimizer, num_users, train_mask, val_mask, y)
    
    # Get recommendations and evaluate
    all_user_probs = get_job_recommendations_batch(model, data)
    
    # Evaluate on test set
    test_mask_np = test_mask.numpy()
    true_jobs_test = df_non_students.loc[test_mask_np, 'DevType'].values
    test_user_indices = np.where(test_mask_np)[0]
    
    predicted_jobs_test = []
    for user_idx in tqdm(test_user_indices, desc="Generating predictions"):
        user_probs = all_user_probs[user_idx]
        recommended_jobs = get_top_n_jobs(user_probs, dev_type_encoder, top_n=10)
        predicted_jobs_test.append(recommended_jobs)
    
    # Calculate and print MRR
    mrr_score_test = calculate_mrr(true_jobs_test, predicted_jobs_test)
    print(f'MRR Score on Test Set: {mrr_score_test}')
    
    # Print some example recommendations
    print("\nExample Recommendations:")
    for i in range(min(5, len(test_user_indices))):
        user_idx = test_user_indices[i]
        true_job = df_non_students.loc[user_idx, 'DevType']
        recommendations = predicted_jobs_test[i]
        print(f"\nUser {i+1}")
        print(f"True Job: {true_job}")
        print(f"Top 5 Recommendations: {', '.join(recommendations[:5])}")

Epoch: 000, Loss: 3.3653, Val Acc: 0.1288
Epoch: 010, Loss: 1.0860, Val Acc: 0.7757
Epoch: 020, Loss: 0.6317, Val Acc: 0.8641
Epoch: 030, Loss: 0.3855, Val Acc: 0.9098
Epoch: 040, Loss: 0.2630, Val Acc: 0.9491
Early stopping at epoch 43


Generating predictions: 100%|██████████| 7903/7903 [00:01<00:00, 4779.21it/s]

MRR Score on Test Set: 0.959217566566042

Example Recommendations:

User 1
True Job: Developer, full-stack
Top 5 Recommendations: Developer, full-stack, Developer, front-end, Designer, Data engineer, Educator

User 2
True Job: Cloud infrastructure engineer
Top 5 Recommendations: Cloud infrastructure engineer, Engineering manager, Developer, QA or test, Developer, embedded applications or devices, Data or business analyst

User 3
True Job: Other (please specify):
Top 5 Recommendations: Other (please specify):, Developer, desktop or enterprise applications, Developer, mobile, Developer, full-stack, Developer, embedded applications or devices

User 4
True Job: Data scientist or machine learning specialist
Top 5 Recommendations: Data scientist or machine learning specialist, Developer, mobile, Developer, front-end, Security professional, Developer, embedded applications or devices

User 5
True Job: Developer, full-stack
Top 5 Recommendations: Developer, full-stack, Developer, front-end, De

In [43]:
def recommend_jobs_for_students(df, model, data, dev_type_encoder, skill_encoder, feature_size=64):
    """
    Recommend jobs for students based on their skills using the trained model.
    
    Args:
        df: Original DataFrame containing all data
        model: Trained HGNNJobRecommender model
        data: HeteroData object used for training
        dev_type_encoder: Fitted LabelEncoder for dev types
        skill_encoder: Fitted LabelEncoder for skills
        feature_size: Size of feature vectors (default: 64)
    
    Returns:
        DataFrame with student data and job recommendations
    """
    # Filter students
    df_students = df.query("DevType == 'Student'").reset_index(drop=True)
    
    if len(df_students) == 0:
        return df_students
    
    # Create feature vectors for students
    num_students = len(df_students)
    student_features = torch.nn.functional.normalize(torch.randn(num_students, feature_size), p=2, dim=1)
    
    # Create student-skill edges
    student_skill_src = []
    student_skill_dst = []
    
    for idx, skills in enumerate(df_students['AllSkillsHaveWorkedWith']):
        if skills:  # Check if skills list is not empty
            # Filter only known skills (those in the encoder)
            known_skills = [skill for skill in skills if skill in skill_encoder.classes_]
            if known_skills:
                skill_indices = skill_encoder.transform(known_skills)
                student_skill_src.extend([idx] * len(skill_indices))
                student_skill_dst.extend(skill_indices)
    
    # Create temporary HeteroData object for students
    student_data = HeteroData()
    
    # Copy existing skill and dev_type data
    student_data['skill'].x = data['skill'].x
    student_data['dev_type'].x = data['dev_type'].x
    
    # Add student features and edges
    student_data['user'].x = student_features
    
    if student_skill_src:  # Only create edges if there are any
        student_data['user', 'has_skill', 'skill'].edge_index = torch.tensor([student_skill_src, student_skill_dst])
        student_data['skill', 'rev_has_skill', 'user'].edge_index = torch.tensor([student_skill_dst, student_skill_src])
    
    # Copy other relevant edges from training data
    student_data['dev_type', 'requires_skill', 'skill'].edge_index = data['dev_type', 'requires_skill', 'skill'].edge_index
    student_data['skill', 'rev_requires_skill', 'dev_type'].edge_index = data['skill', 'rev_requires_skill', 'dev_type'].edge_index
    
    # Get recommendations
    model.eval()
    with torch.no_grad():
        all_student_probs = get_job_recommendations_batch(model, student_data)
    
    # Get top recommendations for each student
    recommendations = []
    recommendation_probs = []
    for idx in range(len(df_students)):
        student_probs = all_student_probs[idx]
        # Get indices of top 5 probabilities
        top_indices = torch.argsort(student_probs, descending=True)[:5]
        # Get the actual probability values
        top_probs = student_probs[top_indices].tolist()
        # Get the job titles
        top_jobs = [dev_type_encoder.inverse_transform([i.item()])[0] for i in top_indices]
        
        recommendations.append(top_jobs)
        recommendation_probs.append(top_probs)
    
    # Add recommendations to DataFrame
    df_students['RecommendedJobs'] = recommendations
    df_students['RecommendationConfidence'] = recommendation_probs
    
    return df_students

# Example usage after training the model:
def add_student_recommendations(df, model, data, dev_type_encoder, skill_encoder):
    """
    Add job recommendations for students to the original DataFrame.
    """
    # Get recommendations for students
    df_students_with_recommendations = recommend_jobs_for_students(
        df, model, data, dev_type_encoder, skill_encoder
    )
    
    # Combine with non-student data
    df_non_students = df.query("DevType != 'Student'").reset_index(drop=True)
    df_non_students['RecommendedJobs'] = None
    df_non_students['RecommendationConfidence'] = None
    
    # Concatenate and sort by original index if needed
    df_combined = pd.concat([df_non_students, df_students_with_recommendations], ignore_index=True)
    
    return df_combined

# Usage in main:
if __name__ == "__main__":    
    # After training the model, add recommendations for students
    df_with_recommendations = add_student_recommendations(
        df, model, data, dev_type_encoder, skill_encoder
    )
    
    # Example of printing recommendations for the first few students
    print("\nStudent Job Recommendations:")
    student_recommendations = df_with_recommendations[
        df_with_recommendations['DevType'] == 'Student'
    ][['RecommendedJobs', 'RecommendationConfidence']].head()
    
    for idx, row in student_recommendations.iterrows():
        print(f"\nStudent {idx + 1}")
        for job, confidence in zip(row['RecommendedJobs'], row['RecommendationConfidence']):
            print(f"- {job}: {confidence:.3f}")


Student Job Recommendations:

Student 54344
- Developer, full-stack: 0.115
- Developer, back-end: 0.112
- Engineering manager: 0.064
- Developer, desktop or enterprise applications: 0.058
- Developer, front-end: 0.056

Student 54345
- Developer, full-stack: 0.191
- Developer, front-end: 0.123
- Developer, back-end: 0.104
- Developer, desktop or enterprise applications: 0.051
- Engineering manager: 0.042

Student 54346
- Developer, front-end: 0.158
- Developer, full-stack: 0.130
- Developer, back-end: 0.114
- Developer, desktop or enterprise applications: 0.056
- Engineering manager: 0.046

Student 54347
- Developer, full-stack: 0.132
- Developer, back-end: 0.103
- Engineering manager: 0.069
- Developer, front-end: 0.064
- Developer, desktop or enterprise applications: 0.062

Student 54348
- Developer, full-stack: 0.134
- Developer, back-end: 0.098
- Developer, front-end: 0.061
- Developer, desktop or enterprise applications: 0.056
- Engineering manager: 0.052


In [48]:
df_with_recommendations[df_with_recommendations['DevType']=='Student']['RecommendedJobs'].apply(lambda x: x[0]).value_counts()

RecommendedJobs
Developer, full-stack                            3720
Developer, back-end                              1351
Developer, front-end                               19
Developer, desktop or enterprise applications      10
Developer, embedded applications or devices         2
Name: count, dtype: int64